In [1]:
#vmenday2

import jax.numpy as jnp
import jax
from jax import random
import matplotlib.pyplot as plt
from jax import lax
import jax.scipy.stats as st

In [18]:
n = 1000000 #Number of simulations
M = 2192 #Number of observation dates
Spot = 32656.70
sigma = 0.216
r = 0.03875
q = 0.01923
T = 2192/365
dt = T/M
ISavg = Spot #(to be replace by the average price of index before pricing date to be captured)


In [2]:

S0 = jnp.ones((1,n))*Spot
S0 =S0[0]

def Prices(S0,M):
  dB = jnp.sqrt(dt) * jax.random.normal(key=random.PRNGKey(M), shape=(1,jnp.size(S0)))[0]
  S0 = S0 * jnp.exp((r-q-0.5*sigma**2)*dt+sigma*dB)
  
  return S0, S0
SF, S_list = lax.scan(Prices, S0, jnp.arange(M))

In [19]:
ISavg = jnp.ones(n)*ISavg
for i in range(5):
  ISavg = S_list[i,:]+ ISavg
ISavg = ISavg/5
FSavg = jnp.zeros(n)
for i in range(2094, 2189):
  FSavg = S_list[i,:]+ FSavg
FSavg = FSavg/(2189-2094)

In [21]:

val_1=(FSavg > 1.32*ISavg)+0

V1 = val_1*(17.08 + (10.68*(FSavg - 1.32*ISavg)/ISavg))

val_2 = ((FSavg > 0.92* ((val_1)-1)*-1*ISavg)+0)*((((val_1)-1)*-1*ISavg*1.32 >= FSavg)+0) 

V2 = val_2*(10 + (17.7*(FSavg - 0.92*ISavg)/ISavg))

val_3 = ((FSavg >= 0.76* ((val_2)-1)*-1*ISavg)+0)*((((val_2)-1)*-1*ISavg*0.92 >= FSavg)+0) 

V3 = val_3*(10 + (15*((FSavg - ISavg)/ISavg+0.08)))

val_4 = jnp.ones(n) - (val_1 + val_2 + val_3)

V4 = val_4*(10 + (10*((FSavg - ISavg)/ISavg)))

V = (V1 + V2 + V3 + V4)
V = V* jnp.exp(-r*T)

jnp.mean(V)

DeviceArray(8.434687, dtype=float32)